
# List of Packages to Download

```python
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install -qU langchain-openai
!pip install --upgrade --quiet  rank_bm25 > /dev/null
!pip install faiss-cpu
!pip install faiss-gpu
```
**Make sure to have an OPEN AI Api key available**
______________________________________________________________________________
 

# 1. Load Data


In [2]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [22]:
from langchain.document_loaders import TextLoader

# Initialize the TextLoader with your file path
loader = TextLoader('formatted_output.txt')

# Load data into a list of documents
data = loader.load()


# 2. Make Ensemble of Retrievers

In [26]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [27]:
Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), persist_directory="./chroma_db")


# 3. Make a RAG pipeline


In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [29]:
from openai import OpenAI
OpenAI.api_key = ''
os.environ["LANGCHAIN_API_KEY"] = "" 
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))


/home/koraspond/Documents/llms-model-training/envs/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [30]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [43]:
question = "how much money they spent on the project"
result = qa_chain({"query": question })

In [44]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

('The context provided does not specify the exact amount of money spent on the '
 'project.')
